In [16]:
"""distance based ccd model"""

import pandas as pd
import numpy as np

E = pd.read_excel('D:/ee-composite/src/sqlserver/data/E/Data_E_1_PC1.xlsx').rename(columns={'pc1':'E'})
U = pd.read_excel('D:/ee-composite/src/sqlserver/data/U/Data_U_1_PC1.xlsx').rename(columns={'pc1':'U'})
E_slope = pd.read_excel('D:/ee-composite/src/sqlserver/data/E/Data_E_1_PC1_slope.xlsx').set_index('sn')
U_slope = pd.read_excel('D:/ee-composite/src/sqlserver/data/U/Data_U_1_PC1_slope.xlsx').set_index('sn')
# print(U_slope,U_slope.shape)

CCD = pd.concat([E,U['U']],axis=1)
# print(CCD,CCD.shape)

e = CCD['E']
u = CCD['U']
CCD['ccd'] = 0.5+0.5*(np.sqrt(e**2+u**2) - np.sqrt((e-1)**2+(u-1)**2))/np.sqrt(2)
# print(CCD,CCD.shape)

ccd_city_mean = CCD.groupby(['sn','city','citycn','province','classname']).mean()
ccd_city_std = CCD.groupby(['sn','city','citycn','province','classname']).std()
ccd_year_mean = CCD.groupby('year').mean()
ccd_year_std = CCD.groupby('year').std()
ccd_province_mean = CCD.groupby(['province','year']).mean()
ccd_classname_mean = CCD.groupby(['classname','year']).mean()
# print(CCD_mean,CCD_mean.shape)

CCD['iy'] = CCD['i']*CCD['ccd']
CCD['i2'] = CCD['i']*CCD['i']
ccd_slope = CCD.groupby(['sn','city','citycn','province','classname']).sum().reset_index()
ccd_slope['year'] = 2020
ccd_slope['slope_ccd'] = (36*ccd_slope['iy']-ccd_slope['i']*ccd_slope['ccd'])/(36*ccd_slope['i2']-ccd_slope['i']*ccd_slope['i'])
ccd_slope=ccd_slope.set_index('sn')
ccd_slope['slope_e'] = E_slope['slope']
ccd_slope['slope_u'] = U_slope['slope']
# print(ccd_slope,ccd_slope.shape)

CCD.to_excel('Data_ccd.xlsx', sheet_name='ccd')
ccd_city_mean.to_excel('Data_ccd_city_mean.xlsx', sheet_name='ccd_mean')
ccd_city_std.to_excel('Data_ccd_city_mean_std.xlsx', sheet_name='ccd_mean_std')
ccd_year_mean.to_excel('Data_ccd_year_mean.xlsx', sheet_name='ccd_mean')
ccd_year_std.to_excel('Data_ccd_year_mean_std.xlsx', sheet_name='ccd_mean_std')
ccd_province_mean.to_excel('Data_ccd_province_mean.xlsx', sheet_name='ccd_mean')
ccd_classname_mean.to_excel('Data_ccd_classname_mean.xlsx', sheet_name='ccd_mean')
ccd_slope.loc[:,['citycn','province','classname','city','slope_ccd','slope_e','slope_u']].to_excel('Data_ccd_slope.xlsx', sheet_name='slope')

ccd1985 = CCD[CCD['year']==1985].loc[:,['sn','city','citycn','classname','classid','ccd','E','U']].reset_index().set_index('sn')
ccd1985['E_U1985'] = ccd1985['E'] - ccd1985['U']
ccd1985['E_U'] = ccd1985['E'] - ccd1985['U']
# print(ccd1985,ccd1985.shape)

ccd2020 = CCD[CCD['year']==2020].loc[:,['sn','city','citycn','classname','classid','ccd','E','U']].reset_index().set_index('sn')
ccd2020['slope_ccd'] = ccd_slope['slope_ccd']
ccd2020['slope_e'] = ccd_slope['slope_e']
ccd2020['slope_u'] = ccd_slope['slope_u']
ccd2020['E_U'] = ccd2020['E'] - ccd2020['U']

#由 U lags  -->> E lags
def my_test(eu1985, eu2020):
    return eu1985>=0 and eu2020<0

ccdU_E_alter = pd.concat([ccd1985.loc[:,['city','classname','classid','E_U1985']],ccd2020.loc[:,['E_U']]],axis=1)
ccdU_E_alter['eu']= ccdU_E_alter.apply(lambda x: my_test(x['E_U1985'],x['E_U']),axis=1)
ccdU_E_rate = ccdU_E_alter.loc[:,['classname','eu','city']].groupby(['classname','eu']).count()
# print(ccdU_E_alter,ccdU_E_alter.shape)
# print('\n不同城市规模 由 U lags  -->> E lags\n',ccdU_E_rate,ccdU_E_rate.shape)


#UD 不协调[0-0.25]  MD一般协调[0.25,0.45]  CD协调[0.45,1.0] ????标准待商榷
ccd2020= ccd2020.assign(flagid=lambda x: pd.cut(x['ccd'],bins=[0, 0.25, 0.45, 1.0],labels=["1", "2", "3"],right=False,include_lowest=True))
ccd2020= ccd2020.assign(lagsid=lambda x: pd.cut(x['E_U'],bins=[-1.0, -0.05, 0.05, 1.0],labels=["1", "3", "2"],right=False,include_lowest=True))
ccd2020['typeid']= (ccd2020['flagid'].astype(str)+ccd2020['lagsid'].astype(str))
# print(ccd2020,ccd2020.shape)
ccd1985= ccd1985.assign(flagid=lambda x: pd.cut(x['ccd'],bins=[0, 0.25, 0.45, 1.0],labels=["1", "2", "3"],right=False,include_lowest=True))
ccd1985= ccd1985.assign(lagsid=lambda x: pd.cut(x['E_U'],bins=[-1.0, -0.05, 0.05, 1.0],labels=["1", "3", "2"],right=False,include_lowest=True))
ccd1985['typeid']= (ccd1985['flagid'].astype(str)+ccd1985['lagsid'].astype(str))
# print(ccd1985,ccd1985.shape)

ccd2020= ccd2020.assign(compare=lambda x: pd.cut(x['E_U'],bins=[-10, 0, 10],labels=['U>E','U<E'],right=False,include_lowest=True))
ccd2020= ccd2020.assign(type   =lambda x: pd.cut(x['typeid'].astype(int),bins=[11,12,13,21,22,23,31,32,33,44],labels=['E1','E2','E3','E4','E5','E6','E7','E8','E9'],right=False,include_lowest=True))
ccd2020= ccd2020.assign(flag   =lambda x: pd.cut(x['flagid'].astype(int),bins=[1,2,3,4],labels=['UD','MD','CD'],right=False,include_lowest=True))
ccd2020= ccd2020.assign(flags  =lambda x: pd.cut(x['typeid'].astype(int),bins=[11,12,13,21,22,23,31,32,33,44],labels=['UD.(E lags)','UD.(U lags)','UD.(E & U are in balanced)','MD.(E lags)','MD.(U lags)','MD.(E & U are in balanced)','CD.(E lags)','CD.(U lags)','CD.(E & U are in balanced)'],right=False,include_lowest=True))
# print('\n2020.城市分类UD MD CD\n',ccd2020, ccd2020.shape)
ccd1985= ccd1985.assign(compare=lambda x: pd.cut(x['E_U'],bins=[-10, 0, 10],labels=['U>E','U<E'],right=False,include_lowest=True))
ccd1985= ccd1985.assign(type   =lambda x: pd.cut(x['typeid'].astype(int),bins=[11,12,13,21,22,23,31,32,33,44],labels=['E1','E2','E3','E4','E5','E6','E7','E8','E9'],right=False,include_lowest=True))
ccd1985= ccd1985.assign(flag   =lambda x: pd.cut(x['flagid'].astype(int),bins=[1,2,3,4],labels=['UD','MD','CD'],right=False,include_lowest=True))
ccd1985= ccd1985.assign(flags  =lambda x: pd.cut(x['typeid'].astype(int),bins=[11,12,13,21,22,23,31,32,33,44],labels=['UD.(E lags)','UD.(U lags)','UD.(E & U are in balanced)','MD.(E lags)','MD.(U lags)','MD.(E & U are in balanced)','CD.(E lags)','CD.(U lags)','CD.(E & U are in balanced)'],right=False,include_lowest=True))
# print('\n1985.城市分类UD MD CD\n',ccd1985, ccd1985.shape)
# ccd2020.to_excel('Data_ccd_type_2020.xlsx', sheet_name='ccdtype')

classcount2020 = ccd2020.loc[:,['classname','flag','city']].groupby(['classname','flag']).count()
classcount1985 = ccd1985.loc[:,['classname','flag','city']].groupby(['classname','flag']).count()
classcompare = ccd2020.loc[:,['classname','compare','city']].groupby(['classname','compare']).count()
print('\n 1985.不同城市规模 UD MD CD\n',classcount1985, classcount1985.shape)
print('\n2020.不同城市规模 UD MD CD\n',classcount2020, classcount2020.shape, '\n2020.比较U<>E的城市个数\n',classcompare, classcompare.shape)

ccdrate = ccd2020.loc[:,['classname','slope_ccd']].groupby(['classname']).mean()
erate = ccd2020.loc[:,['classname','slope_e']].groupby(['classname']).mean()
urate = ccd2020.loc[:,['classname','slope_u']].groupby(['classname']).mean()
ccdratestd = ccd2020.loc[:,['classname','slope_ccd']].groupby(['classname']).std(ddof=0)
eratestd = ccd2020.loc[:,['classname','slope_e']].groupby(['classname']).std(ddof=0)
uratestd = ccd2020.loc[:,['classname','slope_u']].groupby(['classname']).std(ddof=0)
# print('\n1985-2020.不同城市规模ccd斜率\n',ccdrate, ccdrate.shape, '\n1985-2020.不同城市规模eeq斜率\n',erate,erate.shape,'\n1985-2020.不同城市规模u斜率\n',urate,urate.shape)
# print('\n1985-2020.不同城市规模ccd斜率-std\n',ccdratestd, ccdratestd.shape, '\n1985-2020.不同城市规模eeq斜率-std\n',eratestd,eratestd.shape,'\n1985-2020.不同城市规模u斜率-std\n',uratestd,uratestd.shape)

cd2020 = classcount2020.reset_index()
cdpercent2020 = cd2020[cd2020['flag']=='CD'].groupby(['classname']).sum()/cd2020.groupby(['classname']).sum()
cdpercent1985 = cd2020[cd2020['flag']=='CD'].groupby(['classname']).sum()/cd2020.groupby(['classname']).sum()
cdadd2020 = cdpercent2020.reset_index()
print('\n2020.CD城市的比例\n',cdpercent2020,cdpercent2020.shape)


# CCD.groupby(['sn','city','citycn']).mean().to_excel('Data_ccd_citymean.xlsx', sheet_name='PC1_mean')
# CCD.groupby(['sn','city','citycn']).std().to_excel('Data_ccd_citymeanstd.xlsx', sheet_name='PC1_std')


 1985.不同城市规模 UD MD CD
                 city
classname flag      
LC        UD      14
          MD       8
          CD       1
MC        UD       0
          MD       2
          CD       1
SC        UD       7
          MD       8
          CD       0 (9, 1)

2020.不同城市规模 UD MD CD
                 city
classname flag      
LC        UD       1
          MD      15
          CD       7
MC        UD       0
          MD       0
          CD       3
SC        UD       2
          MD       5
          CD       8 (9, 1) 
2020.比较U<>E的城市个数
                    city
classname compare      
LC        U>E        12
          U<E        11
MC        U>E         2
          U<E         1
SC        U>E        11
          U<E         4 (6, 1)

2020.CD城市的比例
                city
classname          
LC         0.304348
MC         1.000000
SC         0.533333 (3, 1)
